In [1]:
from pyscf import gto, scf
import numpy as np
#import matplotlib.pyplot as plt
# from pyscf.geomopt.berny_solver import optimize
from pyscf.grad import rhf as grhf
#from pyscf.hessian import rhf as hrhf
import pyscf.qmmm  
import inspect
from functools import reduce
from pyscf.scf import cphf
angstrom = 1 / 0.52917721067
from pyscf.scf._response_functions import _gen_rhf_response 
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV
def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords(), deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

In [16]:
mol = gto.M(atom='C 0 0 0; O 0 0 1.2', unit="Bohr",basis="sto-3g")
mf = scf.RHF(mol)
e=mf.scf()

converged SCF energy = -108.485919016289
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000    10.3428116554
1 O    -0.0000000000     0.0000000000   -10.3428116554
----------------------------------------------


In [3]:
mol1 = gto.M(atom='C 0 0 0; O 0 0 1.2', unit="Bohr",basis="sto-3g")

In [4]:
mol.atom_charges()

array([6, 8], dtype=int32)

In [5]:
print(inspect.getsource(mol.atom_charges))

    def atom_charges(self):
        '''np.asarray([mol.atom_charge(i) for i in range(mol.natm)])'''
        z = self._atm[:,CHARGE_OF]
        if numpy.any(self._atm[:,NUC_MOD_OF] == NUC_FRAC_CHARGE):
            # Create the integer nuclear charges first then replace the MM
            # particles with the MM charges that saved in _env[PTR_FRAC_CHARGE]
            z = numpy.array(z, dtype=numpy.double)
            idx = self._atm[:,NUC_MOD_OF] == NUC_FRAC_CHARGE
            # MM fractional charges can be positive or negative
            z[idx] = self._env[self._atm[idx,PTR_FRAC_CHARGE]]
        return z



In [6]:
NUC_FRAC_CHARGE=pyscf.gto.mole.NUC_FRAC_CHARGE
PTR_FRAC_CHARGE=pyscf.gto.mole.PTR_FRAC_CHARGE
NUC_MOD_OF=pyscf.gto.mole.NUC_MOD_OF
CHARGE_OF=pyscf.gto.mole.CHARGE_OF
PTR_ZETA=pyscf.gto.mole.PTR_ZETA

print(NUC_FRAC_CHARGE,PTR_FRAC_CHARGE,NUC_MOD_OF,CHARGE_OF)

3 3 2 0


In [7]:
mol._atm

array([[ 6, 20,  1, 23,  0,  0],
       [ 8, 24,  1, 27,  0,  0]], dtype=int32)

In [8]:
mol1._atm[:,NUC_MOD_OF] = NUC_FRAC_CHARGE
mol1._atm

array([[ 6, 20,  3, 23,  0,  0],
       [ 8, 24,  3, 27,  0,  0]], dtype=int32)

In [9]:
mol1._env[mol1._atm[0,PTR_FRAC_CHARGE]]=6.5
mol1._env[mol1._atm[1,PTR_FRAC_CHARGE]]=7.5

In [10]:
mol.atom_charges()

array([6, 8], dtype=int32)

In [11]:
# mol1.build()   !!! never do mol.build(), reset charges to the original values 

In [12]:
mol1.atom_charges()

array([6.5, 7.5])

In [13]:
mol1._atm[:,NUC_MOD_OF] == NUC_FRAC_CHARGE

array([ True,  True])

In [14]:
mol1.built=True
mf1=scf.RHF(mol1)
mf1.scf(dm0=mf.get_init_guess())

converged SCF energy = -105.282904516428


-105.28290451642843

In [15]:
g=mf.Gradients()
g.run()

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000    10.3428116554
1 O    -0.0000000000    -0.0000000000   -10.3428116554
----------------------------------------------


In [16]:
mol1._env[mol1._atm[0,PTR_FRAC_CHARGE]]=6
mol1._env[mol1._atm[1,PTR_FRAC_CHARGE]]=8
mol1.built=True
mf1=scf.RHF(mol1)
mf1.scf(dm0=mf.make_rdm1())
g1=mf1.Gradients()
g1.run()

converged SCF energy = -108.485919016289
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000    -0.0000000000    10.1805341297
1 O     0.0000000000    -0.0000000000   -10.3548125534
----------------------------------------------


In [17]:
mf.scf()

converged SCF energy = -108.48591901629


-108.4859190162895

Praticamente l'energia è corretta, mi da un gradiente, ma qualcosa si sminchia nel calcolo occorre decomporre g nei vari pezzi e quindi , confrontare con quelli della molecola


In [18]:
g.grad_nuc(),g1.grad_nuc()

(array([[  0.        ,   0.        ,  33.33333333],
        [  0.        ,   0.        , -33.33333333]]),
 array([[  0.        ,   0.        ,  33.33333333],
        [  0.        ,   0.        , -33.33333333]]))

In [19]:
a=g1.grad_elec()
a

array([[ 2.29156261e-16, -2.50547906e-16, -2.31527992e+01],
       [ 9.33048392e-17, -9.69739001e-16,  2.29785208e+01]])

In [20]:
b=g.grad_elec()
b

array([[-1.23946665e-15,  7.24698627e-17, -2.29905217e+01],
       [ 1.23946665e-15, -7.24698627e-17,  2.29905217e+01]])

In [21]:
"""
This is the important code ----------------------------------------------------------------

hcore_deriv = mf_grad.hcore_generator(mol)                  dH/dx,dy,dz as a function of (at_idx)
s1 = mf_grad.get_ovlp(mol)%autocall                         dS/dx   
dm0 = mf.make_rdm1(mo_coeff, mo_occ)                          P

vhf = mf_grad.get_veff(mol, dm0)                           P ( d/dx(ml) || ns )

dme0 = mf_grad.make_rdm1e(mo_energy, mo_coeff, mo_occ)        W

atmlst = range(mol.natm)
aoslices = mol.aoslice_by_atom()
de = numpy.zeros((len(atmlst),3))
for k, ia in enumerate(atmlst):
    p0, p1 = aoslices [ia,2:]
    h1ao = hcore_deriv(ia) 
    de[k] += numpy.einsum('xij,ij->x', h1ao, dm0)                                    P*dH/dx
# nabla was applied on bra in vhf, *2 for the contributions of nabla|ket>
    de[k] += numpy.einsum('xij,ij->x', vhf[:,p0:p1], dm0[p0:p1]) * 2           P (Pd/dx(ml||ns))    -as construction on dm0
    de[k] -= numpy.einsum('xij,ij->x', s1[:,p0:p1], dme0[p0:p1]) * 2        W dS/dx

return de
""" 
pass

In [22]:
print(np.allclose(g.get_hcore(),g1.get_hcore()))
print(np.allclose(mf.make_rdm1(),mf1.make_rdm1()))
(P,P1)=(mf.make_rdm1(),mf1.make_rdm1())
print(np.allclose(P,P1))
print(np.allclose( mf.make_rdm1(mf.mo_coeff, mf.mo_occ),mf1.make_rdm1(mf1.mo_coeff, mf1.mo_occ)) )
print(np.allclose(g.get_veff(mol, P),g1.get_veff(mol1, P1)))
print(np.allclose(g.make_rdm1e(mf.mo_energy, mf.mo_coeff, mf.mo_occ),g1.make_rdm1e(mf1.mo_energy, mf1.mo_coeff, mf1.mo_occ) ))
print(np.allclose(g.get_ovlp(mol),g1.get_ovlp(mol1)))
print(np.allclose(mf.get_hcore(mol),mf1.get_hcore(mol1)))
# what else ????????
#print(np.allclose(mf.mo_coeff,mf1.mo_coeff))  # Can accur orbital rotation 
print(np.allclose(g.hcore_generator(mol)(0),g1.hcore_generator(mol1)(0)))
print(np.allclose(g.hcore_generator(mol)(1),g1.hcore_generator(mol1)(1)))

True
True
True
True
True
True
True
True
False
False


In [23]:
g1.hcore_generator(mol1)(1).shape

(3, 10, 10)

In [24]:
np.einsum('xij,ij->x', g1.hcore_generator(mol1)(0), P1)-np.einsum('xij,ij->x', g.hcore_generator(mol)(0), P)

array([ 2.30424570e-14, -1.79665077e-14, -1.62276894e-01])

In [25]:
np.einsum('xij,ij->x', g1.hcore_generator()(1), P1)-np.einsum('xij,ij->x', g.hcore_generator()(1), P)

array([-2.27199959e-14,  1.67462207e-14, -1.20015298e-02])

In [26]:
a-b

array([[ 1.46862291e-15, -3.23017769e-16, -1.62277526e-01],
       [-1.14616181e-15, -8.97269138e-16, -1.20008979e-02]])

In [27]:
"""
h1 = mf.get_hcore(mol)
        def hcore_deriv(atm_id):
            shl0, shl1, p0, p1 = aoslices[atm_id]
            with mol.with_rinv_at_nucleus(atm_id):
                vrinv = mol.intor('int1e_iprinv', comp=3) # <\nabla|1/r|>
                vrinv *= -mol.atom_charge(atm_id)
                if with_ecp and atm_id in ecp_atoms:
                    vrinv += mol.intor('ECPscalar_iprinv', comp=3)
            vrinv[:,p0:p1] += h1[:,p0:p1]
            return vrinv + vrinv.transpose(0,2,1)
    return hcore_deriv
    """

"\nh1 = mf.get_hcore(mol)\n        def hcore_deriv(atm_id):\n            shl0, shl1, p0, p1 = aoslices[atm_id]\n            with mol.with_rinv_at_nucleus(atm_id):\n                vrinv = mol.intor('int1e_iprinv', comp=3) # <\nabla|1/r|>\n                vrinv *= -mol.atom_charge(atm_id)\n                if with_ecp and atm_id in ecp_atoms:\n                    vrinv += mol.intor('ECPscalar_iprinv', comp=3)\n            vrinv[:,p0:p1] += h1[:,p0:p1]\n            return vrinv + vrinv.transpose(0,2,1)\n    return hcore_deriv\n    "

In [28]:
print(np.allclose(mol.aoslice_by_atom(),mol1.aoslice_by_atom()))
np.allclose(mf.get_hcore(),mf1.get_hcore())

True


True

In [29]:
mol.aoslice_by_atom()
vrinv = mol.intor('int1e_iprinv', comp=3) # <\nabla|1/r|>
vrinv *= -mol.atom_charge(0)
vrinv1 = mol1.intor('int1e_iprinv', comp=3) # <\nabla|1/r|>
vrinv1 *= -mol1.atom_charge(0)

In [30]:
np.allclose(vrinv,vrinv1)

True

In [31]:
np.allclose(vrinv + vrinv.transpose(0,2,1),g.hcore_generator(mol)(0) )

False

In [32]:
aoslices = mol.aoslice_by_atom()
h = mf.get_hcore(mol)
shl0, shl1, p0, p1 = aoslices[0]
with mol.with_rinv_at_nucleus(0):
    vrinv = mol.intor('int1e_iprinv', comp=3) # <\nabla|1/r|>
    vrinv *= -mol.atom_charge(0)
vrinv[:,:,p0:p1] += h[:,p0:p1]

In [33]:
 h[:,p0:p1].shape,vrinv[:,:,p0:p1].shape

((10, 5), (3, 10, 5))

In [34]:
np.allclose(vrinv + vrinv.transpose(0,2,1),g.hcore_generator(mol)(0) )

False

In [35]:
g.hcore_generator(mol)

<function pyscf.grad.rhf.hcore_generator.<locals>.hcore_deriv(atm_id)>

In [36]:
print( mol.aoslice_by_atom())
print( mol1.aoslice_by_atom())

[[ 0  3  0  5]
 [ 3  6  5 10]]
[[ 0  3  0  5]
 [ 3  6  5 10]]


In [37]:
print(inspect.getsource(g.hcore_generator))

def hcore_generator(mf, mol=None):
    if mol is None: mol = mf.mol
    with_x2c = getattr(mf.base, 'with_x2c', None)
    if with_x2c:
        hcore_deriv = with_x2c.hcore_deriv_generator(deriv=1)
    else:
        with_ecp = mol.has_ecp()
        if with_ecp:
            ecp_atoms = set(mol._ecpbas[:,gto.ATOM_OF])
        else:
            ecp_atoms = ()
        aoslices = mol.aoslice_by_atom()
        h1 = mf.get_hcore(mol)
        def hcore_deriv(atm_id):
            shl0, shl1, p0, p1 = aoslices[atm_id]
            with mol.with_rinv_at_nucleus(atm_id):
                vrinv = mol.intor('int1e_iprinv', comp=3) # <\nabla|1/r|>
                vrinv *= -mol.atom_charge(atm_id)
                if with_ecp and atm_id in ecp_atoms:
                    vrinv += mol.intor('ECPscalar_iprinv', comp=3)
            vrinv[:,p0:p1] += h1[:,p0:p1]
            return vrinv + vrinv.transpose(0,2,1)
    return hcore_deriv



In [38]:
def hcore_deriv(atm_id,mol=mol,grad=g):
    shl0, shl1, p0, p1 = aoslices[atm_id]
    with mol.with_rinv_at_nucleus(atm_id):
        vrinv = mol.intor('int1e_iprinv', comp=3) # <\nabla|1/r|>
        vrinv *= -mol.atom_charge(atm_id)
    vrinv[:,p0:p1] += grad.get_hcore()[:,p0:p1]
    return vrinv + vrinv.transpose(0,2,1)

In [39]:
hcore_deriv(0).shape,g.hcore_generator(mol)(0).shape

((3, 10, 10), (3, 10, 10))

In [40]:
g.get_hcore().shape

(3, 10, 10)

In [41]:
np.allclose(hcore_deriv(0),g.hcore_generator(mol)(0))

True

In [42]:
np.allclose(g.get_hcore(mol),g1.get_hcore(mol1))

True

In [43]:
np.allclose(hcore_deriv(0),hcore_deriv(0,mol=mol))

True

In [44]:
mol.atom_charge(0)


6

In [45]:
mol1.atom_charge(0)

6.0

In [46]:
atm_id=1
with mol.with_rinv_at_nucleus(atm_id):
    vrinv = mol.intor('int1e_iprinv', comp=3) # <\nabla|1/r|>
with mol1.with_rinv_at_nucleus(atm_id):
    vrinv1 = mol1.intor('int1e_iprinv', comp=3) # <\nabla|1/r|>      mol.intor('int1e_iprinv', comp=3) questo differisce fra mol e mol1

In [47]:
np.allclose(vrinv,vrinv1)

False

In [48]:
np.allclose(mol.intor('int1e_iprinv', comp=3),mol1.intor('int1e_iprinv', comp=3))
mol1.intor('int1e_iprinv', comp=3).shape

(3, 10, 10)

In [49]:
with mol1.with_rinv_at_nucleus(atm_id) and mol.with_rinv_at_nucleus(atm_id) :
    print(np.allclose(mol.intor('int1e_iprinv', comp=3),mol1.intor('int1e_iprinv', comp=3)))

False


In [50]:
np.allclose(mol1.intor('int1e_iprinv', comp=3),vrinv)

False

In [51]:
mol1._env[mol1._atm[atm_id,PTR_ZETA]]

8.0

In [52]:
class FracMole(gto.Mole):
    def with_rinv_at_nucleus(self, atm_id):
        rinv = self.atom_coord(atm_id)
        self._env[pyscf.gto.mole.AS_RINV_ORIG_ATOM] = atm_id  # required by ecp gradients
        return self.with_rinv_origin(rinv)

In [53]:
mol2=FracMole()
mol2.build(atom='C 0 0 0; O 0 0 1.2', unit="Bohr",basis="sto-3g")

In [54]:
mol2._atm[:,NUC_MOD_OF] = NUC_FRAC_CHARGE
mol2._atm

array([[ 6, 20,  3, 23,  0,  0],
       [ 8, 24,  3, 27,  0,  0]], dtype=int32)

In [55]:
mol2._atm[:,NUC_MOD_OF] == NUC_FRAC_CHARGE
mol2._env[mol2._atm[0,pyscf.gto.mole.PTR_FRAC_CHARGE]]=6.
mol2._env[mol2._atm[1,pyscf.gto.mole.PTR_FRAC_CHARGE]]=8.
mf2 = scf.RHF(mol2)
e=mf2.scf(dm0=P)

converged SCF energy = -108.48591901629


In [56]:
mol2.atom_charges()

array([6., 8.])

In [57]:
g2=mf2.Gradients()

In [58]:
g2.run()

--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000    10.3428116496
1 O     0.0000000000    -0.0000000000   -10.3428116496
----------------------------------------------


In [59]:
g.run()

--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000    10.3428116554
1 O     0.0000000000    -0.0000000000   -10.3428116554
----------------------------------------------


In [60]:
mol3=FracMole()
mol3.build(atom='C 0 0 0; O 0 0 1.2', unit="Bohr",basis="sto-3g")
mol3._atm[:,NUC_MOD_OF] = NUC_FRAC_CHARGE
mol3._atm[:,NUC_MOD_OF] == NUC_FRAC_CHARGE
mol3._env[mol3._atm[0,pyscf.gto.mole.PTR_FRAC_CHARGE]]=6.3
mol3._env[mol3._atm[1,pyscf.gto.mole.PTR_FRAC_CHARGE]]=7.7
mf3= scf.RHF(mol3)
e=mf3.scf(dm0=mf3.init_guess_by_atom()) #SAD guess XD

converged SCF energy = -106.483078849107


In [61]:
g3=mf3.Gradients()
g3.run()

--------------- RHF gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000    10.1915287716
1 O     0.0000000000    -0.0000000000   -10.1915287716
----------------------------------------------


In [62]:
compcalc=fc(mf,[.3,-.3])
compcalc.scf()

converged SCF energy = -106.483078849106


-106.48307884910648

In [63]:
print(dir(mol))

['__add__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_suffix', '_atm', '_atom', '_bas', '_basis', '_built', '_ecp', '_ecpbas', '_env', '_keys', '_libcint_ctr_coeff', '_nelectron', '_pseudo', '_symm_axes', '_symm_orig', 'ao2mo', 'ao_labels', 'ao_loc', 'ao_loc_2c', 'ao_loc_nr', 'aoslice_2c_by_atom', 'aoslice_by_atom', 'aoslice_nr_by_atom', 'apply', 'atom', 'atom_charge', 'atom_charges', 'atom_coord', 'atom_coords', 'atom_mass_list', 'atom_nelec_core', 'atom_nshells', 'atom_pure_symbol', 'atom_shell_ids', 'atom_symbol', 'bas_angular', 'bas_atom', 'bas_coord', 'bas_ctr_coeff', 'bas_exp', 'bas_kappa', 'bas_len_cart', 'bas_len_spinor', 'bas_nctr', 'bas_nprim', 'b

In [64]:
len([[1,2,3,4,5],[1,2,3,4,2]])

2

In [65]:
from FracChargMole import FcM

In [66]:
fcm=FcM(fcs=[.05,-.05],atom="C 0 0 0; O 0 0 1.2",unit="Bohrs")

[6 8]
[6 8]


In [67]:
mffc=scf.RHF(fcm)
mffc.scf(dm0=mffc.init_guess_by_atom())

converged SCF energy = -108.135250204437


-108.13525020443662

In [68]:
fcm.atom_charges()

array([6.05, 7.95])

In [75]:
hs=mf.Hessian()

In [76]:
hs.kernel()

array([[[[-8.61900970e+00,  7.88956472e-16, -2.27024875e-16],
         [ 3.51327825e-16, -8.61900970e+00,  1.76279053e-16],
         [-2.27024874e-16,  1.76279050e-16,  3.43907703e+01]],

        [[ 8.61900970e+00, -3.23572250e-16,  2.27024874e-16],
         [-7.88956472e-16,  8.61900970e+00, -1.76279050e-16],
         [ 2.27024875e-16, -1.76279053e-16, -3.43907703e+01]]],


       [[[ 8.61900970e+00, -7.88956472e-16,  2.27024875e-16],
         [-3.23572250e-16,  8.61900970e+00, -1.76279053e-16],
         [ 2.27024874e-16, -1.76279050e-16, -3.43907703e+01]],

        [[-8.61900970e+00,  2.40497399e-16, -2.27024875e-16],
         [ 7.38931113e-16, -8.61900970e+00,  1.76279053e-16],
         [-2.27024874e-16,  1.76279050e-16,  3.43907703e+01]]]])

In [73]:
help(hs)

Help on Hessian in module pyscf.hessian.rhf object:

class Hessian(pyscf.lib.misc.StreamObject)
 |  Hessian(scf_method)
 |  
 |  Non-relativistic restricted Hartree-Fock hessian
 |  
 |  Method resolution order:
 |      Hessian
 |      pyscf.lib.misc.StreamObject
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, scf_method)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  gen_hop(hobj, mo_energy=None, mo_coeff=None, mo_occ=None, verbose=None)
 |  
 |  get_hcore(self, mol=None)
 |  
 |  hcore_generator(self, mol=None)
 |  
 |  hess = kernel(self, mo_energy=None, mo_coeff=None, mo_occ=None, atmlst=None)
 |  
 |  hess_elec(hessobj, mo_energy=None, mo_coeff=None, mo_occ=None, mo1=None, mo_e1=None, h1ao=None, atmlst=None, max_memory=4000, verbose=None)
 |  
 |  hess_nuc(self, mol=None, atmlst=None)
 |  
 |  kernel(self, mo_energy=None, mo_coeff=None, mo_occ=None, atmlst=None)
 |      Kernel function is the main driver of a method. 